# Classificador com corpus de brinquedo

Vamos modelizar um conjunto fictício de resenhas de clientes sobre determinado restaurante. Além dos textos, temos à disposição uma etiqueta para cada resenha informando se a opinião do cliente soa positiva ou negativa.

A partir das probabilidades calculadas, será possível gerar uma função que avalie novas resenhas, classificando-as como positivas ou negativas.

In [1]:
import os
import zipfile

In [2]:
for f in os.listdir('Enron/spam'):
    print(f)

3087.2004-12-06.GP.spam.txt
2192.2004-09-16.GP.spam.txt
2296.2004-09-27.GP.spam.txt
0184.2004-01-11.GP.spam.txt
2229.2004-09-22.GP.spam.txt
3367.2005-01-03.GP.spam.txt
1801.2004-08-11.GP.spam.txt
1993.2004-08-26.GP.spam.txt
4252.2005-04-13.GP.spam.txt
4827.2005-07-07.GP.spam.txt
4643.2005-06-05.GP.spam.txt
2803.2004-11-12.GP.spam.txt
0674.2004-03-19.GP.spam.txt
2813.2004-11-14.GP.spam.txt
1858.2004-08-17.GP.spam.txt
4211.2005-04-08.GP.spam.txt
4286.2005-04-18.GP.spam.txt
4306.2005-04-18.GP.spam.txt
2447.2004-10-09.GP.spam.txt
1821.2004-08-13.GP.spam.txt
4955.2005-08-04.GP.spam.txt
3610.2005-01-28.GP.spam.txt
3176.2004-12-14.GP.spam.txt
3661.2005-02-02.GP.spam.txt
4935.2005-07-27.GP.spam.txt
4428.2005-05-01.GP.spam.txt
1926.2004-08-20.GP.spam.txt
4262.2005-04-14.GP.spam.txt
0153.2004-01-07.GP.spam.txt
5002.2005-08-13.GP.spam.txt
4799.2005-07-04.GP.spam.txt
4505.2005-05-17.GP.spam.txt
2530.2004-10-18.GP.spam.txt
0104.2003-12-29.GP.spam.txt
2480.2004-10-12.GP.spam.txt
1066.2004-05-07.GP.s

1660.2004-07-19.GP.spam.txt
1850.2004-08-16.GP.spam.txt
0733.2004-03-26.GP.spam.txt
1205.2004-05-26.GP.spam.txt
3695.2005-02-04.GP.spam.txt
4120.2005-03-21.GP.spam.txt
4283.2005-04-16.GP.spam.txt
1816.2004-08-13.GP.spam.txt
2242.2004-09-22.GP.spam.txt
2472.2004-10-10.GP.spam.txt
0997.2004-05-02.GP.spam.txt
3573.2005-01-25.GP.spam.txt
1936.2004-08-20.GP.spam.txt
0799.2004-04-09.GP.spam.txt
2338.2004-09-30.GP.spam.txt
3018.2004-12-02.GP.spam.txt
0110.2003-12-30.GP.spam.txt
4039.2005-03-14.GP.spam.txt
1753.2004-08-03.GP.spam.txt
2603.2004-10-24.GP.spam.txt
0803.2004-04-10.GP.spam.txt
3465.2005-01-13.GP.spam.txt
4698.2005-06-14.GP.spam.txt
1343.2004-06-14.GP.spam.txt
1260.2004-06-04.GP.spam.txt
1338.2004-06-12.GP.spam.txt
3787.2005-02-10.GP.spam.txt
0332.2004-01-29.GP.spam.txt
2301.2004-09-27.GP.spam.txt
0854.2004-04-15.GP.spam.txt
3818.2005-02-13.GP.spam.txt
5068.2005-08-23.GP.spam.txt
4958.2005-08-04.GP.spam.txt
1450.2004-06-29.GP.spam.txt
0805.2004-04-11.GP.spam.txt
4497.2005-05-17.GP.s

In [3]:
# Estas são as resenhas para a modelização:
corpus = [('Esse restaurante é um lixo', 'NEG'), ('A comida servida no inferno', 'NEG'),
          ('Lugar imundo', 'NEG'), ('Perfeito para o seu cachorro', 'POS'), ('Restaurante ótimo', 'POS')]

# Esta é a resenha com a qual vamos testar o classificador. Você pode tentar com outras, também.
teste = 'Restaurante horroroso. A comida é lixo em estado coloidal! O outro emprego do garçom é carcereiro de masmorra.'

**Pré-processamento:**

*   Tokenização
*   Limpeza
* Filtragem de stop words

In [4]:
import itertools
from collections import Counter
from numpy import prod
from math import log

import nltk
nltk.download('punkt')
from nltk import tokenize 
nltk.download('stopwords')
stops = nltk.corpus.stopwords.words('portuguese')


def tokenizar(str_texto):
    return tokenize.word_tokenize(str_texto, language='portuguese')


def sem_stops(lst_palavras):
    return [p for p in lst_palavras if p not in stops]


def limpar(lista):
    return [i.lower() for i in lista if i.isalpha()]


def pre_processar(str_texto):
    return sem_stops(limpar(tokenizar(str_texto)))


def achatar(lista):
    return list(itertools.chain(*lista))

[nltk_data] Downloading package punkt to
[nltk_data]     /home/brunorosilva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/brunorosilva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
corpus = [(pre_processar(i[0]), i[1]) for i in corpus]
corpus

[(['restaurante', 'lixo'], 'NEG'),
 (['comida', 'servida', 'inferno'], 'NEG'),
 (['lugar', 'imundo'], 'NEG'),
 (['perfeito', 'cachorro'], 'POS'),
 (['restaurante', 'ótimo'], 'POS')]

**Separação dos documentos**

*   Extração de duas listas separadas contendo documentos etiquetados como "negativos" e "positivos"
*   Extração do vocabulário total do corpus (juntando as palavras das resenhas positivas e das negativas). Atenção! Lembre-se de que o vocabulário são as palavras sem repetição.


In [6]:
negativos = [i[0] for i in corpus if i[1] == 'NEG']
positivos = [i[0] for i in corpus if i[1] == 'POS']

vocab = set(achatar(negativos)) | set(achatar(positivos))

**Contagens**

*   *N* documentos negativos
*   *N* documentos positivos
* *N* total de documentos

* *N* itens no vocabulário
* Contagens de ocorrências de cada palavra nas resenhas negativas. **Dica:** use a função `Counter()` do módulo `collection`s para isso.
* Contagens de ocorrências de cada palavra nas resenhas positivas. 
* *N* total de palavras "negativas"
* *N* total de palavras "positivas"



In [7]:
# Contagens de documentos (resenhas)
n_docs_negativos = len(negativos)
n_docs_positivos = len(positivos)
n_docs_total = n_docs_negativos + n_docs_positivos

# N itens no vocabulário
n_vocab = len(vocab)

# Contagens de cada palavra (dicionários de ocorrências)
tokens_neg = achatar(negativos)
cont_neg = Counter(tokens_neg)
tokens_pos = achatar(positivos)
cont_pos = Counter(tokens_pos)

# N total de palavras em cada classe
n_tokens_neg = sum(cont_neg.values())
n_tokens_pos = sum(cont_pos.values())

**Cálculo das probabilidades de classificação**

Há duas probabilidades a calcular, uma para cada classe ($c$): a probabilidade de uma resenha qualquer ser negativa e a probabilidade de ela ser falsa.

Essa probabilidade é dada por:

\begin{equation}
P(c) \prod\limits_{i=1}^n P(f_{i} | c)
\end{equation}

Onde a probabilidade de cada atributo $f$ é dada pelo número de ocorrências (tokens) desse atributo dividido pelo número de tokens em cada classe.

**Dica:** para calcular o produtório de uma lista, use a função `prod` do módulo `numpy`.


Vamos suavizar a classificação com o método de Laplace, isto é, somando 1 a cada atributo no numerador e somando também a cardinalidade do vocabulário ao denominador.

A probabilidade isolada (anterior) da classe $c$ é dada por:

\begin{equation}
	P(c) = \dfrac{contagem(c)}{N}
\end{equation}

Já a probabilidade suavizada de um atributo qualquer pertencer a $c$ é:

\begin{equation}
	P(f_{i} | c) = \dfrac{contagem(f_{i},c) + 1}{contagem(c) + V}
 \end{equation}

Por fim, para evitar o underflow aritmético, é sempre uma boa ideia calcular probabilidades encadeadas com logaritmos. A diferença com documentos e vocabulários pequenos não é grande, mas tende a ser quando se trabalha com dados reais.

\begin{equation}
\hat{c} = \underset{c \in \mathcal{C}}{\operatorname{argmax}} \ \log P(c) + \sum\limits_{i=1}^n \log P(f_{i} | c)
\end{equation}

Agora, com base nessas informações, procure classificar a mensagem-teste apresentada acima.

In [8]:
tokens_teste = pre_processar(teste)
tokens_teste = [i for i in tokens_teste if i in vocab]

# Com logs
prob_neg = log(n_docs_negativos / n_docs_total) + sum([log((cont_neg[i] + 1) / (n_tokens_neg + n_vocab)) for i in tokens_teste])
prob_pos = log(n_docs_positivos / n_docs_total) + sum([log((cont_pos[i] + 1) / (n_tokens_pos + n_vocab)) for i in tokens_teste])

print(teste)
print(prob_neg, prob_pos)
if prob_neg > prob_pos:
    print('O cliente não gostou do restaurante.')
else:
    print('O cliente gostou do restaurante.')


Restaurante horroroso. A comida é lixo em estado coloidal! O outro emprego do garçom é carcereiro de masmorra.
-6.931024114254803 -8.140315540159985
O cliente não gostou do restaurante.


---

# **Tarefa:** Detecção de spam com corpus de dados reais

Nessa tarefa, você vai trabalhar com parte do corpus de mensagens de e-mail da Enron.

As mensagens estão em arquivos de texto curtos em inglês (principalmente, mas não só) e têm anotações manuais no título e na primeira linha de texto com sua etiqueta como "spam" (1) ou "ham" (0). 

Aqui está uma mensagem de exemplo:


---


0

Subject: meter 1517 - jan 1999
george ,
i need the following done :
jan 13
zero out 012 - 27049 - 02 - 001 receipt package id 2666
allocate flow of 149 to 012 - 64610 - 02 - 055 deliv package id 392
jan 26
zero out 012 - 27049 - 02 - 001 receipt package id 3011
zero out 012 - 64610 - 02 - 055 deliv package id 392
these were buybacks that were incorrectly nominated to transport contracts
( ect 201 receipt )
let me know when this is done
hc

---

Observe o 0 na primeira linha. Ele indica que a mensagem foi etiquetada como *ham*.

Você deve:



1.   Abrir cada arquivo de texto.
2.   Pré-processar os dados, implementando os seguintes procedimentos:

*   Tokenização
* Eliminação de stop words
*   Limpeza e homogeneização dos tokens
* Stemização

Use o NLTK para gerar a lista de stop words em inglês e para implementar um stemizador também em inglês:

---

```
import nltk
stops = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

def stemizar(lista):
    return [stemmer.stem(i) for i in lista]
```

---

3. Dividir o corpus pré-processado em treinamento (80%) e teste (20%).

4. Usando o corpus de treinamento, realizar as contagens do vocabulário, das classes (*spam* ou *ham*) e dos atributos (cada token corresponde a um atributo).

5. Calcular as probabilidades relacionadas às contagens.

6. Implementar uma função bayes() que receba uma mensagem e devolva a probabilidade de ela ser classificada como *spam* e como *ham*:

`return prob_spam, prob_ham`

7. Nessa função, não deixe de incluir:

* Uma condicional para testar se as palavras a classificar fazem parte do vocabulário de treinamento. As que não fazem devem ser simplesmente ignoradas (ficar de fora do cálculo).

* A suavização de Laplace.

7. Classificar todo o corpus de teste passando cada mensagem pela função bayes().

8. Avaliar a performance do classificador: para cada mensagem, comparar a classificação com as etiquetas de *spam* ou *ham* e gerar uma lista com os resultados dessa avaliação em termos de Verdadeiro Positivo (VP), Verdadeiro Negativo (VN), Falso Positivo (FP) e Falso Negativo (FN).

9. Com base nessa lista, calcular:

* precisao = vp / (vp + fp)
* cobertura = vp / (vp + fn)
* acuracia = (vp + vn) / (vp + vn + fp + fn)
* Medida_F = 2 * (precisao * cobertura) / (precisao + cobertura)








# Preprocessing

In [34]:
import nltk
stops = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
import itertools
from collections import Counter
from numpy import prod
from math import log

from nltk import tokenize 
nltk.download('stopwords')
stops = nltk.corpus.stopwords.words('english')




def tokenizar(str_texto):
    return tokenize.word_tokenize(str_texto, language='english')

def sem_stops(lst_palavras):
    return [p for p in lst_palavras if p not in stops]

def limpar(lista):
    return [i.lower() for i in lista if i.isalpha()]

def pre_processar(str_texto):
    return sem_stops(stemizar(limpar(tokenizar(str_texto))))

def achatar(lista):
    return list(itertools.chain(*lista))

def stemizar(lista):
    return [stemmer.stem(i) for i in lista]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/brunorosilva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
import random
random.seed(42)

In [64]:
from tqdm.notebook import tqdm

In [65]:
train = []
test  = []

file_list = os.listdir("Enron")

random.shuffle(file_list)

for f in tqdm(file_list):
    fl = open("Enron/"+f, 'r')
    arquivo_full = fl.read()
    fl.close()
    label = arquivo_full[0]
    texto = arquivo_full[1:]
    texto_processado = pre_processar(texto)
    
    if random.random() > 0.8:
        fw = open("Enron_preprocessed/test/"+f, "w")
        fw.write(label + "\n" + str(texto_processado))
        fw.close()

        tupla_atual = (texto_processado, label)
        test.append(tupla_atual)
    else:
        fw = open("Enron_preprocessed/train/"+f, "w")
        fw.write(label + "\n" + str(texto_processado))
        fw.close()

        tupla_atual = (texto_processado, label)
        train.append(tupla_atual)

  0%|          | 0/5157 [00:00<?, ?it/s]

In [67]:
negativos = [i[0] for i in train if i[1] == '0']
positivos = [i[0] for i in train if i[1] == '1']

vocab = set(achatar(negativos)) | set(achatar(positivos))

In [68]:
# Contagens de documentos (resenhas)
n_docs_negativos = len(negativos)
n_docs_positivos = len(positivos)
n_docs_total = n_docs_negativos + n_docs_positivos

# N itens no vocabulário
n_vocab = len(vocab)

# Contagens de cada palavra (dicionários de ocorrências)
tokens_neg = achatar(negativos)
cont_neg = Counter(tokens_neg)
tokens_pos = achatar(positivos)
cont_pos = Counter(tokens_pos)

# N total de palavras em cada classe
n_tokens_neg = sum(cont_neg.values())
n_tokens_pos = sum(cont_pos.values())

In [58]:
tokens_teste = pre_processar(teste)
tokens_teste = [i for i in tokens_teste if i in vocab]

# Com logs
prob_neg = log(n_docs_negativos / n_docs_total) + sum([log((cont_neg[i] + 1) / (n_tokens_neg + n_vocab)) for i in tokens_teste])
prob_pos = log(n_docs_positivos / n_docs_total) + sum([log((cont_pos[i] + 1) / (n_tokens_pos + n_vocab)) for i in tokens_teste])

print(teste)
print(prob_neg, prob_pos)
if prob_neg > prob_pos:
    print('O cliente não gostou do restaurante.')
else:
    print('O cliente gostou do restaurante.')

Restaurante horroroso. A comida é lixo em estado coloidal! O outro emprego do garçom é carcereiro de masmorra.
-20.76214229132958 -17.870642672749554
O cliente gostou do restaurante.
